# Paso 0: Configuración y Carga
Primero, necesitamos cargar las librerías que usaremos y el df_master que creamos en el notebook anterior.

Importante: Para guardar tu df_master al final del Notebook 01, te recomiendo usar to_parquet() en lugar de .to_csv(). Es mucho más rápido y mantiene los tipos de datos (como fechas) correctamente.

In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [12]:
# --- PASO 0: Conectar Google Drive ---
# ¡DEBES ejecutar esto en CADA notebook, CADA vez que lo abras!

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [13]:

# Configuración de Seaborn para gráficos más bonitos
sns.set_style("whitegrid")

# --- Cargar Rutas (igual que antes) ---
BASE_DIR = "/content/drive/My Drive/AI Projects/Customer Churn/"
DATA_DIR = BASE_DIR + "datasets/"

# --- Cargar el DataFrame Maestro ---
print("Cargando DataFrame Maestro...")
try:
    # Cargar desde Parquet (mucho más rápido)
    df = pd.read_parquet(DATA_DIR + "df_master_transaccional.parquet")
    print("¡DataFrame cargado exitosamente!")
except FileNotFoundError:
    print("Error: No se encontró 'df_master_transaccional.parquet'.")
    print("Por favor, ejecuta el Notebook 01 y asegúrate de guardar el archivo.")

display(df.head())

Cargando DataFrame Maestro...
¡DataFrame cargado exitosamente!


,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,total_payment_value,num_items,total_product_value,total_freight_value,product_id,product_category_name_english
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00,7c396fd4830fd04220f754e42b4e5bff,3149,sao paulo,SP,38.71,1.0,29.99,8.72,87285b34884572647811a353c7ac498a,housewares
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00,af07308b275d755c9edb36a90c618231,47813,barreiras,BA,141.46,1.0,118.70,22.76,595fac2a385ac33a80bd5114aec74eb8,perfumery
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00,3a653a41f6f9fc3d2a113cf8398680e8,75265,vianopolis,GO,179.12,1.0,159.90,19.22,aa4383b373c6aca5d8797843e5594415,auto
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15 00:00:00,7c142cf63193a1473d2e66489a9ae977,59296,sao goncalo do amarante,RN,72.20,1.0,45.00,27.20,d0b61bfb1de832b15ba9d266ca96e5b0,pet_shop
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26 00:00:00,72632f0f9dd73dfee390c9b22eb56dd6,9195,santo andre,SP,28.62,1.0,19.90,8.72,65266b2da20d04dbe00c5c2d3bb7859e,stationery


# Paso 1: Limpieza y Preparación Crítica
No podemos analizar el churn basándonos en pedidos "cancelados". Solo nos importan los pedidos completados. También convertimos las fechas a formato datetime para poder calcular el tiempo.

In [14]:
# --- 1.1: Filtrar solo pedidos entregados ---
# Un cliente "activo" es uno que completa una compra.
print(f"Filas antes del filtro: {len(df)}")

# Guardamos los pedidos 'delivered' en una copia para evitar warnings
df_delivered = df[df['order_status'] == 'delivered'].copy()
print(f"Filas después de filtrar por 'delivered': {len(df_delivered)}")


# --- 1.2: Convertir columnas de fecha ---
# (Si usaste parquet, es posible que ya estén correctas, pero esto lo asegura)
print("\nConvirtiendo columnas de fecha...")
df_delivered['order_purchase_timestamp'] = pd.to_datetime(df_delivered['order_purchase_timestamp'])
df_delivered['order_delivered_customer_date'] = pd.to_datetime(df_delivered['order_delivered_customer_date'])

print("Tipos de datos de fechas:")
print(df_delivered[['order_purchase_timestamp', 'order_delivered_customer_date']].info())

Filas antes del filtro: 99441
Filas después de filtrar por 'delivered': 96478

Convirtiendo columnas de fecha...
Tipos de datos de fechas:
<class 'pandas.core.frame.DataFrame'>
Index: 96478 entries, 0 to 99440
Data columns (total 2 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   order_purchase_timestamp       96478 non-null  datetime64[ns]
 1   order_delivered_customer_date  96470 non-null  datetime64[ns]
dtypes: datetime64[ns](2)
memory usage: 2.2 MB
None


# Paso 2: Definición de Churn (El Corazón del Proyecto)
Aquí es donde creamos nuestro dataset de clientes y definimos la variable objetivo (y).

2.1: Definir la Fecha de "Hoy" (Snapshot Date)
Como los datos son históricos, simulamos que "hoy" es un día después del último pedido registrado.

In [15]:
# --- 2.1: Definir la Snapshot Date ---
# Tomaremos la fecha del último pedido
max_date = df_delivered['order_purchase_timestamp'].max()

# Esta será nuestra fecha de referencia "hoy" para el análisis.
snapshot_date = max_date + pd.Timedelta(days=1)
print(f"Nuestra fecha para el análisis es: {snapshot_date}")

Nuestra fecha para el análisis es: 2018-08-30 15:00:37


## 2.2: Colapsar de "Pedidos" a "Clientes" (Análisis RFM)
Este es el groupby() más importante. Pasamos de un dataset de transacciones a uno de clientes

In [16]:
# --- 2.2: Crear el DataFrame de Clientes (RFM inicial) ---
print("\nAgrupando pedidos por cliente...")

# Usamos 'customer_unique_id' para agrupar
df_clientes = df_delivered.groupby('customer_unique_id').agg(

    # 1. (R) Recencia: La fecha del ÚLTIMO pedido
    last_purchase_date=('order_purchase_timestamp', 'max'),

    # 2. (F) Frecuencia: El NÚMERO de pedidos únicos
    frequency=('order_id', 'nunique'),

    # 3. (M) Monetario: La SUMA de todo lo que han pagado
    monetary=('total_payment_value', 'sum')

).reset_index() # .reset_index() convierte el groupby en un DataFrame

display(df_clientes.head())


Agrupando pedidos por cliente...


,customer_unique_id,last_purchase_date,frequency,monetary
0,0000366f3b9a7992bf8c76cfdf3221e2,2018-05-10 10:56:27,1,141.90
1,0000b849f77a49e4a4ce2b2a4ca5be3f,2018-05-07 11:11:27,1,27.19
2,0000f46a3911fa3c0805444483337064,2017-03-10 21:05:03,1,86.22
3,0000f6ccb0745a6a4b88665a16c9f078,2017-10-12 20:29:41,1,43.62
4,0004aac84e0df4da2b147fca70cf8255,2017-11-14 19:45:42,1,196.89


## 2.3: Calcular la Recencia y Definir el Churn
Ahora que tenemos la last_purchase_date de cada cliente, calculamos los días que han pasado y aplicamos nuestro umbral de negocio.

In [17]:
# --- 2.3: Calcular Recencia y Crear la variable 'churn' ---

# 1. Calcular Recencia (días desde la última compra)
# snapshot_date - last_purchase_date nos da un 'Timedelta', .dt.days lo convierte a número
df_clientes['recency'] = (snapshot_date - df_clientes['last_purchase_date']).dt.days

# 2. Definir el Umbral de Churn
# ¡Esta es tu gran decisión de negocio!
# ¿Cuánto tiempo sin comprar es "churn"? Usemos 180 días (6 meses)
UMBRAL_DIAS_CHURN = 180

# 3. Crear la variable objetivo (y)
# Si recency > 180 días -> 1 (Churn)
# Si recency <= 180 días -> 0 (Activo)
df_clientes['churn'] = np.where(df_clientes['recency'] > UMBRAL_DIAS_CHURN, 1, 0)

print("\n--- DataFrame de Clientes (con Churn) ---")
display(df_clientes.head())

# 4. Revisar la distribución de tu variable objetivo
print("\nDistribución de Churn vs. Activos:")
print(df_clientes['churn'].value_counts())
print("\nDistribución de Churn vs. Activos (Porcentaje):")
print(df_clientes['churn'].value_counts(normalize=True))


--- DataFrame de Clientes (con Churn) ---


,customer_unique_id,last_purchase_date,frequency,monetary,recency,churn
0,0000366f3b9a7992bf8c76cfdf3221e2,2018-05-10 10:56:27,1,141.90,112,0
1,0000b849f77a49e4a4ce2b2a4ca5be3f,2018-05-07 11:11:27,1,27.19,115,0
2,0000f46a3911fa3c0805444483337064,2017-03-10 21:05:03,1,86.22,537,1
3,0000f6ccb0745a6a4b88665a16c9f078,2017-10-12 20:29:41,1,43.62,321,1
4,0004aac84e0df4da2b147fca70cf8255,2017-11-14 19:45:42,1,196.89,288,1



Distribución de Churn vs. Activos:
churn
1    55252
0    38106
Name: count, dtype: int64

Distribución de Churn vs. Activos (Porcentaje):
churn
1    0.591829
0    0.408171
Name: proportion, dtype: float64


In [19]:
df_clientes.to_parquet(DATA_DIR + "df_clientes_rfm.parquet")
# Al final del Notebook 02, ejecuta esto:
print("Guardando df_clientes...")

# Asegúrate de que 'df_clientes' existe antes de guardarlo
display(df_clientes.head())

df_clientes.to_parquet(DATA_DIR + "df_clientes_rfm.parquet")

print(f"¡Archivo 'df_clientes_rfm.parquet' guardado en {DATA_DIR}")

Guardando df_clientes...


,customer_unique_id,last_purchase_date,frequency,monetary,recency,churn
0,0000366f3b9a7992bf8c76cfdf3221e2,2018-05-10 10:56:27,1,141.90,112,0
1,0000b849f77a49e4a4ce2b2a4ca5be3f,2018-05-07 11:11:27,1,27.19,115,0
2,0000f46a3911fa3c0805444483337064,2017-03-10 21:05:03,1,86.22,537,1
3,0000f6ccb0745a6a4b88665a16c9f078,2017-10-12 20:29:41,1,43.62,321,1
4,0004aac84e0df4da2b147fca70cf8255,2017-11-14 19:45:42,1,196.89,288,1


¡Archivo 'df_clientes_rfm.parquet' guardado en /content/drive/My Drive/AI Projects/Customer Churn/datasets/
